In [ ]:
"""
**OPTIONAL/IGNORE**

First we need to import some stuff. Feel free to ignore this cell.

If you're interested, each import has an associated comment that explains why
the import is useful/necessary.
"""

# Used for checking how many cores are available for processing.
import multiprocessing
# Used for constructing paths.
from pathlib import Path
# Used to work out how much time the processing took.
from time import time

# Essential for all mathematical operations we'll be carrying out.
import numpy as np

# diffraction_utils is a library developed at Diamond by Richard Brearton
# (richard.brearton@diamond.ac.uk) to ease the task of parsing data files and
# carrying out some common calculations. Here, we'll be using it to define
# frames of reference, and parse nexus files.
from diffraction_utils import Frame, I07Nexus

# The following imports are required for the core of the calculation code, also
# written by Richard Brearton (richard.brearton@diamond.ac.uk).
# These functions are used to calculate the region of reciprocal space that was
# sampled during the scan, and to calculate the step size we should use to
# achieve the file size requested.
from fast_rsm.meta_analysis import get_step_from_filesize
# This is the central Experiment object, which stores all the logic related to
# mapping the experiment.
from fast_rsm.experiment import Experiment


In [ ]:
"""
**ESSENTIAL**

This cell requires action! Make sure you set all of the variables defined here.
"""

# What was your scattering geometry/how was your sample mounted? Options are
# 'horizontal', 'vertical' and 'DCD'.
setup = 'vertical'

# The experiment number, used to work out where your data is stored.
experiment_number = 'si31695-1'

# The sub-directory containing your experimental data. Leave as None if unused.
# Otherwise, if the data was stored in a subdirectory called "day_1", e.g.
#   /dls/i07/data/2022/si32333-1/day_1/
# then you should use:
#   data_sub_directory = "day_1"
data_sub_directory = "Ag100clean/"

# The year the experiment took place.
year = 2022

# The scan numbers of the scans that we want to use to produce this reciprocal
# space map. For example, the default value of scan_numbers shows how to specify
# every scan between number 421772 and 421778 inclusive, but skipping scan
# number 421776.
scan_numbers = [445500, 445501, 445502]

# Uncomment the following to set scan_numbers equal to every scan number between
# scan_start and scan_stop (inclusive of scan_stop):
# scan_start = 439168
# scan_stop = 439176
# scan_numbers = list(range(scan_start, scan_stop + 1))

# The beam centre, as can be read out from GDA, in pixel_x, pixel_y. If your
# map looks wacky, you probably cocked this up.
beam_centre = (242, 95)

# The distance between the sample and the detector (or, if using the DCD, the
# distance between the receiving slit and the detector). Units of meters.
detector_distance = 930e-3

# The frame/coordinate system you want the map to be carried out in.
# Options for frame_name argument are:
#     Frame.hkl     (map into hkl space - requires UB matrix in nexus file)
#     Frame.sample_holder   (standard map into 1/Å)
#     Frame.lab     (map into frame attached to lab. I dont think you want this)
#
# Options for coordinates argument are:
#     Frame.cartesian   (normal cartesian coords: hkl, Qx Qy Qz, etc.)
#     Frame.polar       (cylindrical polar with cylinder axis along l/Qz)
#
# Frame.polar will give an output like a more general version of PyFAI.
# Frame.cartesian is for hkl maps and Qx/Qy/Qz. Any combination of frame_name
# and coordinates will work, so try them out; get a feel for them.
frame_name = Frame.hkl
coordinates = Frame.cartesian

# Ignore pixels with an intensity below this value. If you don't want to ignore
# any pixels, then set min_intensity = None. This is useful for dynamically
# creating masks (which, in retrospect... might not actually be useful).
min_intensity = None

# How large would you like your output file to be, in MB? 100MB normally gives
# very good resolution without sacrificing performance. If you want something
# higher resolution, feel free, but be aware that the performance of the map and
# the analysis will start to suffer above around 1GB.
# Max file size is 2GB (2000MB).
output_file_size = 100


In [ ]:
"""
**OPTIONAL/IGNORE**
This cell prepares the calculation. You probably shouldn't change anything here
unless you know what you're doing.
"""

# Which synchrotron axis should become the out-of-plane (001) direction.
# Defaults to 'y'; can be 'x', 'y' or 'z'.
if setup == 'vertical':
    oop = 'x'
elif setup == 'horizontal':
    oop = 'y'
elif setup == 'DCD':
    oop = 'y'
else:
    raise ValueError(
        "Setup not recognised. Must be 'vertical', 'horizontal' or 'DCD.")

if output_file_size > 2000:
    raise ValueError("output_file_size must not exceed 2000. "
                     f"Value received was {output_file_size}.")

# Max number of cores available for processing.
num_threads = multiprocessing.cpu_count()

# Work out where the data is.
data_dir = Path(f"/dls/i07/data/{year}/{experiment_number}/")
# data_dir = Path(f"/Users/richard/Data/i07/{experiment_number}/")

# Store this for later.
processing_dir = data_dir / "processing"
if data_sub_directory is not None:
    data_dir /= Path(data_sub_directory)

# You can make this what you like, but note that same datetime data will be
# inserted to ensure that your output file has a unique name.
save_name = "mapped_scan_"

# Work out the paths to each of the nexus files. Store as pathlib.Path objects.
nxs_paths = [data_dir / f"i07-{x}.nxs" for x in scan_numbers]

# Construct the Frame object from the user's preferred frame/coords.
map_frame = Frame(frame_name=frame_name, coordinates=coordinates)


In [ ]:
"""
**REQUIRED**

This cell contains all of the logic for running the calculation. You shouldn't
run this on your local computer, it'll either raise an exception or take
forever.

Uncomment the routines that you would like to run.
"""


if __name__ == "__main__":
    # We always need to prepare the experiment.
    experiment = Experiment.from_i07_nxs(
        nxs_paths, beam_centre, detector_distance, setup)

    # There's a hot pixel that needs masking. Comment this out if you don't want
    # to mask anything.
    experiment.mask_pixels((83, 233))

    # These example nexus data files are broken. The data needs to be backed up
    # from .dat files, which contains the motor positions. Comment this out in
    # the likely event that you don't need to do this.
    for i, scan in enumerate(experiment.scans):
        dat_path = data_dir / data_dir / f"{scan_numbers[i]}.dat"
        scan.metadata.data_file.populate_data_from_dat(dat_path)

    # Uncomment to run and save a binned reciprocal space map.
    # experiment.binned_reciprocal_space_map(
    #     num_threads, map_frame, output_file_size=output_file_size, oop=oop)

    # Uncomment to get intensity vs L.
    # intensity, l = experiment.intensity_vs_l(num_threads, oop=oop)

    # Uncomment to get intensity vs |Q|.
    # intensity, q = experiment.intensity_vs_q(
    #     num_threads, oop=oop, num_bins=1000)
    
    # Uncomment to get intensity vs 2theta.
    # intensity, tth = experiment.intensity_vs_tth(
    #     num_threads, oop=oop, num_bins=1000)

    # Finally, print that it's finished We'll use this to work out when the
    # processing is done.
    print("PROCESSING FINISHED.")

class DontContinue(Exception):
    """Raise to stop processing at this cell"""


raise DontContinue()


In [ ]:
"""
**ESSENTIAL**

This is the cell that you should execute to run this notebook on the cluster.

DO NOT EXECUTE THIS MULTIPLE TIMES. IT WILL SUBMIT MULTIPLE JOBS TO THE CLUSTER.
PLEASE BE RESPONSIBLE.
"""

# We need this to grab the current working directory.
import os

# We'll need this to run the program that will submit the cluster job.
# This module isn't needed for the calculation itself, which is why it is
# imported here.
import subprocess

# First, we save this as "map.py". Make sure it doesn't already exist.
try:
    os.remove("map.py")
except OSError:
    pass

# Convert this notebook to a python script in our home directory.
!jupyter nbconvert --to script map.ipynb

# Submit the job, which in turn loads the Hamilton module and runs:
# qsub -pe smp 40 -l m_mem_free=2.5G -P i07 cluster_job.sh
subprocess.run(["sh", "/dls_sw/apps/fast_rsm/current/scripts/cluster_sub.sh"])


In [ ]:
"""
The following cells contain convenience tools for e.g. interacting with and
visualising data.
"""


In [ ]:
"""
**IGNORE**

This cell just defines a couple of handy functions. Feel free to ignore.
"""

def most_recent_cluster_output():
    """
    Returns the filename of the most recent cluster stdout output.
    """
    # Get all the cluster job files that have been created.
    files = [x for x in os.listdir() if x.startswith("cluster_job.sh.o")]
    numbers = [int(x[16:]) for x in files]

    # Work out which cluster job is the most recent.
    most_recent_job_no = np.max(numbers)
    most_recent_file = ""
    for file in files:
        if str(most_recent_job_no) in file:
            most_recent_file = file

    return most_recent_file


def most_recent_cluster_error():
    """
    Returns the filename of the most recent cluster stderr output.
    """
    # Get all the cluster job files that have been created.
    files = [x for x in os.listdir() if x.startswith("cluster_job.sh.e")]
    numbers = [int(x[16:]) for x in files]

    # Work out which cluster job is the most recent.
    most_recent_job_no = np.max(numbers)
    most_recent_file = ""
    for file in files:
        if str(most_recent_job_no) in file:
            most_recent_file = file

    return most_recent_file


In [ ]:
"""
Are we nearly there yet?

Executing this cell tells you if your most recent cluster submission has
finished executing.
"""

import os
import numpy as np

# Get all the cluster job files that have been created.
files = [x for x in os.listdir() if x.startswith("cluster_job.sh.o")]
numbers = [int(x[16:]) for x in files]

# Work out which cluster job is the most recent.
most_recent_job_no = np.max(numbers)
most_recent_file = ""
for file in files:
    if str(most_recent_job_no) in file:
        most_recent_file = file

# Open that file, and see if it ends in "Finished."
with open(most_recent_file, 'r') as f:
    lines = f.read().splitlines()
    
    # Check if there's nothing in the file yet.
    if len(lines) == 0:
        print("Processing either not started or crashed.")
    last_line = lines[-1].strip()
    if last_line.startswith("PROCESSING FINISHED."):
        print("Most recent processing has completed.")
    else:
        print("Most recent processing has not yet completed.")
    
